# Differential expression analysis

This workbook performs differential expression analysis at cell type level. 
It prepares the data for a pseudo-bulk differential expression analysis (part 1) and 
performs a DE analysis at the single-cell level as well (part2). Pseudobulk DE results will also be computed. The analysis was run on the batch corrected version (per individual) of the workflow. 

In [ ]:
import besca as bc
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from scipy import sparse, io
import os
import time
import logging
import pkg_resources
import seaborn as sns
import itertools
import sys
sc.logging.print_versions()

# for standard processing, set verbosity to minimum
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80)
version = '2.4'
start0 = time.time()

In [ ]:
### Functions to be added to besca

def getMeans(adata, groupval):
    ### Obtain average and fraction expression per group
    ### Gets adata object and groups specific as input
    gene_ids=adata.raw.var.index.values
    clusters=adata.obs[groupval].cat.categories
    obs=adata.raw[:,gene_ids].X.toarray()
    obs=pd.DataFrame(obs,columns=gene_ids,index=adata.obs[groupval])
    average_obs=obs.groupby(level=0).mean()
    obs_bool=obs.astype(bool)
    fraction_obs=obs_bool.groupby(level=0).sum()/obs_bool.groupby(level=0).count()
    return(average_obs,fraction_obs)

def formatmean(average_obs, fraction_obs, what, mycond, myg):
    ### Transforms average and fraction expression to pd for plotting
    df=pd.DataFrame([list(average_obs[myg]),list(fraction_obs[myg]),mycond]).transpose()
    df.index=average_obs.index
    df.columns=[myg+'Avg',myg+'Fract',what]
    return(df)
    
def prepare_df(myg, adata, cond1, cond2, cond3):
    myit1=np.sort(list(set(adata.obs[cond1])))
    myit2=np.sort(list(set(adata.obs[cond2])))
    df=pd.DataFrame()
    for i in myit1:
        cdata=adata[adata.obs[cond1]==i].copy()
        myit3=np.sort(list(set(cdata.obs[cond3])))
        for j in myit3:
            data=cdata[cdata.obs[cond3]==j].copy()
            average_obs,fraction_obs=getMeans(data,cond2)
            mylen=len(fraction_obs[myg])
            df1=pd.DataFrame(data={'Avg':average_obs[myg],'Fct': fraction_obs[myg],
                                   cond1: [i]*mylen,cond3: [j]*mylen,
                                   cond2: fraction_obs[myg].index})
            df=pd.concat([df,df1],ignore_index=True)
    return df

In [ ]:
#define standardized filepaths based on above input
root_path = os.getcwd()
bescapath_full = os.path.dirname(bc.__file__)
bescapath = os.path.split(bescapath_full)[0]

analysis_name = 'sw_besca2'
species='human' ## or mouse for now

results_folder = os.path.join(root_path, 'analyzed/',analysis_name)
results_file = os.path.join(results_folder, analysis_name + '.annotated.h5ad')
results_file_raw = os.path.join(results_folder, analysis_name + '.raw.h5ad')
results_folder_out = results_folder+ '/DE/sc_wilcoxon/'
results_folder_pseudo = results_folder+ '/DE/Pseudobulk/'

clusters='leiden'
split_condition='readout_id' #'experiment' is generally a good default ### change to sampleID
exportPseudobulk=False
exportPseudobulkRaw=False
correctCellNrs=True

In [ ]:
adata = sc.read_h5ad(results_file)
adata


In [ ]:
origdata=adata.copy()

### Output for pseudobulk DE analysis

Iterate over all experiments in split_condition (individual_id in this case) and sum norm counts to pd.Dataframe. Note that this can be done at different levels - over all cells or over specific cell types or clusters. 

#### Save for pseudobulk DE analysis separate on the individual cell types at distinct levels

Manual selection of cell groups or select all members at a certain annotation level

In [ ]:
set(adata.obs['celltype1'])

In [ ]:
#adata=adata[adata.obs['CONDITION']!='T_24_Tumor_C_24_Tumor'].copy() 
#adata=adata[adata.obs['celltype1'].isin(['T cell'])].copy()
#initialsubset='Tcells'
initialsubset='All'
fullsubset='All'

In [ ]:
#adata.obs['celltypevar']=adata.obs['dblabel'].copy()
#adata.obs['celltypevar']=list(adata.obs['celltypevar'].replace('BMP7-positive effector CD8-positive, alpha-beta T cell', 'mature CD8-positive, alpha-beta T cell'))
#adata.obs['celltypevar']=list(adata.obs['celltypevar'].replace('granulocyte', 'monocyte or granulocyte'))

In [ ]:
results_folder+'/DE/Pseudobulk/'

In [ ]:
len(list(adata.obs.columns))

In [ ]:
#'percent_mito', 'n_counts', 'n_genes', 
todrop=['CELL', 'input.path', 
        'leiden', 'celltype0', 'celltype1', 'celltype2', 
        'celltype3', 'dblabel']+list(adata.obs.columns[17:189])


In [ ]:
#'percent_mito', 'n_counts', 'n_genes', 
todrop=['CELL', 'input.path', 
        'leiden', 'celltype0', 'celltype1', 'celltype2', 
        'celltype3','celltype1_co', 'celltype2_co', 'celltype3_co',
        'celltype','celltypevar', 'dblabel']+list(adata.obs.columns[17:187])
exportPseudobulk=False
if (exportPseudobulk==True):
    pseudulk_celltype2=bc.export.pseudobulk(adata, outpath = results_folder_pseudo,
             column = 'celltype2',label  = 'celltype2',
             split_condition  = split_condition,todrop=todrop)
    pseudulk_celltype2=bc.export.pseudobulk(adata, outpath = results_folder_pseudo,
             column = 'celltype2_co',label  = 'celltype2_co',
             split_condition  = split_condition,todrop=todrop)
    pseudulk_celltype2=bc.export.pseudobulk(adata, outpath = results_folder_pseudo,
             column = 'celltype1',label  = 'celltype1',
             split_condition  = split_condition,todrop=todrop)
    pseudulk_celltype2=bc.export.pseudobulk(adata, outpath = results_folder_pseudo,
             column = 'celltype1_co',label  = 'celltype1_co',
             split_condition  = split_condition,todrop=todrop)
    
    pseudulk_celltype=bc.export.pseudobulk(adata, outpath = results_folder_pseudo,
             column = 'dblabel',label  = 'dblabel',
             split_condition  = split_condition,todrop=todrop)


In [ ]:
import sys

In [ ]:
condition='CONDITION'
if (exportPseudobulk==True):
    #for tocount in ['celltype1','celltype2','celltype3', 'celltype', 'celltypevar','celltypevar2','celltypevar3','leiden']:
    for tocount in ['celltype2_co','celltype1_co','celltype2','celltype1','dblabel']:
        ### Or get the numbers
        df=bc.tl.count_occurrence_subset_conditions(adata, count_variable = tocount, 
                        subset_variable =split_condition, condition_identifier = condition,  
                                                    return_percentage = False)
        df.columns=[x.split(' ')[1] for x in list(df.columns)]
        #myresults_folder_out='/pstore/data/biomics/ONC/70206_FAP_CD40/scSEQ_invivo/Tumor/analyzed/sw_besca24_hvgsample/DE/'
        df.to_csv(results_folder_pseudo+'CellCounts-'+tocount+'.tsv', sep='\t')


In [ ]:
#adata=adata[adata.obs['CONDITION']!='T_24_Tumor_C_24_Tumor'].copy() 
adata=adata[adata.obs['IFNGresp'].isin(['IFNGpos'])].copy()
#initialsubset='Tcells'
initialsubset='IFNGpos'
fullsubset='IFNGpos'

In [ ]:
cellorder=[
     'hematopoietic multipotent progenitor cell',
    'plasmacytoid dendritic cell',
    'naive B cell',
 'memory B cell',
 'plasma cell',
    'B myeloid cell doublet',
     'classical monocyte',
        'inflammatory monocyte',
  'monocyte',
     'non-classical monocyte',
 'CD1c-positive myeloid dendritic cell',
     'granulocyte',
 'CD56-bright cytokine secreting natural killer cell',
    
'cytotoxic CD56-dim natural killer cell',
 'CD8-positive, alpha-beta cytotoxic T cell',
     'effector memory CD8-positive, alpha-beta T cell',
    'gamma delta T cell',
 'mucosal invariant T cell',
 'naive thymus-derived CD8-positive, alpha-beta T cell',
    'naive thymus-derived CD4-positive, alpha-beta T cell',
 'central memory CD4-positive, alpha-beta T cell',
     'regulatory T cell',
 'proliferating T cell']

### Plot genes of interest

In [ ]:
#goi=[MCP-1, MIP-1a,  MIP-1b,  IL-8, IP-10, IL-1b, TNF-a, IFNg, IL-6, IL-2 and IL-10]
goi=['CCL2','CCL3','CCL4','CXCL8', 'CXCL10', 'IL1B','TNF', 'IFNG','IL6','IL2','IL10']
goiorder=['CXCL9', 'CXCL10', 'CXCL11','CCL7', 'CCL8','IL6','IL27','IL1RN','IL15','CCL2','SPP1','IL1R1',
 'CCL3','CCL4','CXCL8','IL6R','IL1B','TNF','IL10', 'IFNG', 'IL2','TNFSF13B', 'TNFSF10','CCL5','GZMK','GZMB','FASLG','IGHG1','IGKC']
goi=goiorder
goi=[x.upper() for x in goi]
sc.pl.dotplot(adata,var_names=goi,groupby='dblabel', categories_order=cellorder)

In [ ]:
sc.pl.dotplot(adata[adata.obs['compound_name']=='AAV8'],var_names=goi,groupby='dblabel', 
              dot_max=0.6, vmax=2, categories_order=cellorder)

In [ ]:
sc.pl.dotplot(adata[adata.obs['compound_name']=='Control'],var_names=goi,groupby='dblabel', 
              dot_max=0.6, vmax=2, categories_order=cellorder)

In [ ]:
adata.uns['log1p'] = {'base' : None} # Fix for bug related to scanpy version scverse/scanpy#2239

In [ ]:
set(adata.obs['CONDITION'])

In [ ]:
set(adata.obs['celltype2'])

In [ ]:
plt.rcParams["figure.figsize"] = (6,3)
sc.pl.violin(adata[adata.obs['celltype2'].isin(['CD8-positive, alpha-beta T cell'])],keys=['GZMA'],groupby='CONDITION', rotation=90)

## Differential expression analysis (at invididual cell level)

In [ ]:
condition='CONDITION'


In [ ]:
set(adata.obs['celltype2'])

### Changes in specific cells at specific times

In [ ]:
#results_folder_out = os.path.join(root_path, 'analyzed', analysis_name+'/DE/sc_wilcoxon/')
#results_folder_out = os.path.join(root_path, 'analyzed', analysis_name+'/DE/sc_wilcoxon/48h/')

In [ ]:
#set(adata.obs[''])celltype1

In [ ]:
##mysubset='target cell'
mysubset='natural killer cell'
mysubset='myeloid dendritic cell'
#mysubset='monocyte'
#mysubset='T cell'
mylevel="celltype2_co"
cdata=adata[adata.obs[mylevel]==mysubset].copy()
#cdata=adata[adata.obs[mylevel].isin(['inflammatory monocyte','granulocyte'])].copy()
#cdata=adata[adata.obs[mylevel].isin(['lymphocyte of B lineage','myeloid leukocyte'])].copy() #'CD141-positive myeloid dendritic cell'
cdataorig=origdata[origdata.obs[mylevel]==mysubset].copy()

#mysubset='Monocyte'
#mysubset='NKcell'
mysubset='cDC'

In [ ]:
condition='CONDITION'

In [ ]:
results_out_cell=results_folder_out+'/'+mysubset.replace(" ", "_")+'/'
if not os.path.exists(results_out_cell): os.mkdir(results_out_cell)
if not os.path.exists(results_out_cell+'enrichr/'): os.mkdir(results_out_cell+'enrichr/')

In [ ]:
split_condition

In [ ]:
condition

In [ ]:
mylevel

In [ ]:
cdata.obs

In [ ]:
### Explore the nr. cells per donor - avoid large donor-specific biases
cellnrs=bc.tl.count_occurrence_subset_conditions(cdata, subset_variable = 'sample_id', 
                                                 count_variable = mylevel, condition_identifier = condition,  return_percentage = False)


In [ ]:
plt.rcParams["figure.figsize"] = (12,3)
ax=cellnrs.transpose().plot.bar()
ax.set_ylabel("Nr. cells per sample")

In [ ]:
### Explore the nr. cells per donor - avoid large donor-specific biases
cellnrs=bc.tl.count_occurrence_subset_conditions(cdata, subset_variable = split_condition, 
                                                 count_variable = mylevel, condition_identifier = condition,  return_percentage = False)


In [ ]:
plt.rcParams["figure.figsize"] = (12,3)
ax=cellnrs.transpose().plot.bar()
ax.set_ylabel("Nr. cells per sample")

In [ ]:
cellnrs

In [ ]:
correctCellNrs

In [ ]:
#### Subsample to account for distinct cell nrs. per patient
if correctCellNrs==True:
    myindex=[]
    mylen=[]
    myp=list(cdata.obs[split_condition].cat.categories)
    for i in myp:
        temp=cdata[cdata.obs[split_condition]==i].copy()
        mylen.append(len(temp))
        sc.pp.subsample(temp,n_obs=int(np.min([len(temp.obs),cellnrs.transpose().mean()[0]])))
        myindex.append(temp.obs.index.values)

    flatten=lambda l: [item for myindex in l for item in myindex]
    cdata=cdata[flatten(myindex)].copy()

In [ ]:
### Explore the nr. cells per donor - avoid large donor-specific biases
cellnrs=bc.tl.count_occurrence_subset_conditions(cdata, subset_variable = split_condition, 
                                                 count_variable = mylevel, condition_identifier = condition,  return_percentage = False)

plt.rcParams["figure.figsize"] = (12,3)
ax=cellnrs.transpose().plot.bar()
ax.set_ylabel("Nr. cells per sample")

#### Plot genes of interest

In [ ]:
mypval=0.05
myfc=np.log2(1.5)

In [ ]:
condition

In [ ]:
conds=list(cdata.obs[condition].cat.categories)

In [ ]:
conds

In [ ]:
### Perform DE cells of each celltype3 vs. all other cells
DEgenes=bc.tl.dge.get_de(cdata,condition,demethod='wilcoxon',topnr=5000, logfc=myfc,padj=mypval)


In [ ]:
### Could also check pairwise comparisons if needed
AllComparisons=list(itertools.combinations(conds,2))

In [ ]:
AllComparisons

In [ ]:
#mypairs=[AllComparisons[1],AllComparisons[3],AllComparisons[6],AllComparisons[8],AllComparisons[9]]
#mypairs=AllComparisons
#mypairs=[AllComparisons[0],AllComparisons[1],AllComparisons[3],
#         AllComparisons[6],AllComparisons[9],AllComparisons[14],AllComparisons[15],AllComparisons[16],AllComparisons[17]]

mypairs=[AllComparisons[3],
         AllComparisons[6],AllComparisons[9],AllComparisons[14],AllComparisons[16],AllComparisons[20]]

In [ ]:
mypairs

In [ ]:
mypairs=[('AAV8_1h', 'Control_1h'),
 ('AAV8_4h', 'Control_4h'),
 ('AAV8_24h', 'Control_24h'),
 ('AAV8_4h', 'AAV8_24h'),
('Control_4h', 'Control_24h'),
 ('Control', 'Control_4h')]

In [ ]:
DElist={}
for i in mypairs:
    DElist[i[0]+'-'+i[1]]=bc.tl.dge.get_de(cdata[cdata.obs[condition].isin(i)],condition,demethod='wilcoxon',topnr=5000, logfc=myfc,padj=mypval)


In [ ]:
DElist.keys() #['TLR7ago_antiPDL1_antiVEGF-veh_antiPDL1_antiVEGF']

In [ ]:
DElist.keys()

In [ ]:
mypvalstr=str(mypval).replace('.','')[1:3]
myfcstr=str(myfc).replace('.','')[0:2]

In [ ]:
denrs={}
for i in mypairs:
    print(i[0]+' vs. '+i[1])
    print('DE.1')
    print(DElist[i[0]+'-'+i[1]][i[0]].sort_values('Log2FC', ascending=False).iloc[0:20,:])
    print('GOIs:')
    print(list(set(DElist[i[0]+'-'+i[1]][i[0]]['Name']).intersection(set(goiorder))))
    print('DE.2')
    print(DElist[i[0]+'-'+i[1]][i[1]].sort_values('Log2FC', ascending=False).iloc[0:20,:])
    print('GOIs:')
    print(list(set(DElist[i[0]+'-'+i[1]][i[1]]['Name']).intersection(set(goiorder))))
    denrs[i[0]+'-'+i[1]]=[len(DElist[i[0]+'-'+i[1]][i[0]].index),len(DElist[i[0]+'-'+i[1]][i[1]].index)]
    DElist[i[0]+'-'+i[1]][i[0]].to_csv(results_out_cell+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-pairwise-'+i[0]+'-'+i[1]+'.1.csv')
    DElist[i[0]+'-'+i[1]][i[1]].to_csv(results_out_cell+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-pairwise-'+i[0]+'-'+i[1]+'.2.csv')
denrs=pd.DataFrame(denrs)
denrs.index=['DE1','DE2']

In [ ]:
plt.rcParams["figure.figsize"] = (4,4)
ax=denrs.transpose().plot.bar()
ax.set_ylabel("Nr. pairwise DE genes")

In [ ]:
for i in conds:
    DEgenes[i].to_csv(results_out_cell+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-'+i+'.csv')

In [ ]:
DEgenes.keys()

In [ ]:
set(cdata.obs['sample_id'])

In [ ]:
korder=['D1_Predose','D2_Predose','D3_Predose','D4_Predose','D5_Predose','D6_Predose','D1_1h_ctrl','D2_1h_ctrl',
        'D1_4h_ctrl','D2_4h_ctrl','D3_4h_ctrl','D4_4h_ctrl','D5_4h_ctrl','D6_4h_ctrl',
       'D1_24h_ctrl','D2_24h_ctrl','D3_24h_ctrl','D4_24h_ctrl','D5_24h_ctrl','D6_24h_ctrl',
        'D1_1h_AAV8','D2_1h_AAV8','D1_4h_AAV8','D2_4h_AAV8','D3_4h_AAV8','D4_4h_AAV8','D5_4h_AAV8','D6_4h_AAV8',
       'D1_24h_AAV8','D2_24h_AAV8','D3_24h_AAV8','D4_24h_AAV8','D5_24h_AAV8','D6_24h_AAV8',]

In [ ]:
adata.obs['sample_id']

In [ ]:
sc.pl.matrixplot(cdataorig,list(DEgenes['AAV8_1h'].sort_values('Log2FC',ascending=False).iloc[0:60,]['Name']), 
                 groupby='sample_id', standard_scale='var',
                 categories_order=korder)

In [ ]:
sc.pl.matrixplot(cdataorig,list(DEgenes['AAV8_4h'].sort_values('Log2FC',ascending=False).iloc[0:60,]['Name']), 
                 groupby='sample_id', standard_scale='var',categories_order=korder)

In [ ]:
latei=list(DEgenes['AAV8_24h'].sort_values('Log2FC',ascending=False).iloc[0:20,]['Name'])

In [ ]:
sc.pl.dotplot(cdataorig[cdataorig.obs['compound_name']=='AAV8'],var_names=latei,groupby='dblabel', 
              dot_max=0.6, vmax=2)

sc.pl.dotplot(cdataorig[cdataorig.obs['compound_name']=='Control'],var_names=latei,groupby='dblabel', 
              dot_max=0.6, vmax=2)

In [ ]:
sc.pl.matrixplot(cdataorig,list(DEgenes['AAV8_24h'].sort_values('Log2FC',ascending=False).iloc[0:60,]['Name']), 
                 groupby='sample_id', standard_scale='var',categories_order=korder)

In [ ]:
for key in DEgenes.keys():
    print(key)
    sc.pl.matrixplot(cdataorig,list(DEgenes[key].sort_values('Log2FC',ascending=False).iloc[0:60,]['Name']),
                 groupby='sample_id', standard_scale='var',dendrogram=False, 
                 save="TopDE_"+key+"_"+mysubset.replace(" ", "_")+".pdf")

In [ ]:
denrsall={}
plotlist=[]
plotlist_small=[]
upsets={}
for i in conds:
    print(i)
    print(DEgenes[i].sort_values('Log2FC',ascending=False).iloc[0:15,:])
    denrsall[i]=len(DEgenes[i].index)
    plotlist=plotlist+list(DEgenes[i].sort_values('Log2FC',ascending=False).iloc[0:20,0])
    plotlist_small=plotlist_small+list(DEgenes[i].sort_values('Log2FC',ascending=False).iloc[0:5,0])
    upsets[i]=list(DEgenes[i]['Name'])
denrsall=pd.Series(denrsall)

In [ ]:
plt.rcParams["figure.figsize"] = (6,4)
ax=denrsall.plot.bar()
ax.set_ylabel("Nr. one-vs-others DE genes")

In [ ]:
tops=plotlist
tops=list(dict.fromkeys(tops))

In [ ]:
tops

In [ ]:

sc.pl.dotplot(cdataorig, var_names=tops,groupby=condition)

In [ ]:
sc.pl.heatmap(cdataorig, tops, groupby=condition, 
              swap_axes=True,standard_scale='var',dendrogram=True, cmap='viridis') #vmin=-3, vmax=3,

In [ ]:
sc.pl.matrixplot(cdataorig,tops, groupby='sample_id', standard_scale='var',categories_order=korder)

In [ ]:
#goi=['Cd274','Tlr7','Cd40','Isg15','Mx1','Oas3','Ifi44','Irf7','H2-Q6','Etv6','Ly6a','Psmb9','Lamp1','Ccl5','Atp1b1','Siglech','Ccr5']

#goi=['Cd274','Tlr7','Cd40','Ccr7','Isg15','Mx1','Oas3','Oas1g','Ifi44','Irf7','H2-Q6','Rsad2','Ifi44','Ifit2', 'Ifng']

sc.pl.heatmap(cdataorig, goiorder, groupby=condition, swap_axes=True,standard_scale='var',figsize=[12,6],dendrogram=True, cmap='viridis') #vmin=-3, vmax=3,

In [ ]:
sc.pl.matrixplot(cdataorig,goiorder, groupby='sample_id', standard_scale='var',categories_order=korder)

In [ ]:
sc.pl.dotplot(cdataorig,goiorder, groupby='sample_id',categories_order=korder)

In [ ]:
sc.pl.matrixplot(cdataorig,goiorder, groupby='CONDITION', standard_scale='var',dendrogram=False)

In [ ]:
sc.pl.dotplot(cdataorig,goiorder, groupby='CONDITION', dendrogram=False, dot_max=0.6, vmax=1.5)

In [ ]:
plotlist_small=list(dict.fromkeys(plotlist_small))

In [ ]:
sc.pl.heatmap(cdataorig, plotlist_small, groupby=condition, swap_axes=True,standard_scale='var',figsize=[18,10],dendrogram=True) #vmin=-3, vmax=3,

#### Top DEs for condition 1

In [ ]:
for i in conds[1:len(conds)]: 
    print(i)
    fig = plt.figure(figsize=(12,5))
    bc.pl.gene_expr_split(cdata[cdata.obs[condition].isin([conds[0],i])], genes = list(DEgenes[conds[0]]['Name'][0:40]), split_variable=condition) 

#### Top DE for condition 2

In [ ]:
for i in [conds[0]]+conds[2:len(conds)]: 
    print(i)
    fig = plt.figure(figsize=(12,5))
    bc.pl.gene_expr_split(cdata[cdata.obs[condition].isin([conds[1],i])], genes = list(DEgenes[conds[1]]['Name'][0:40]), split_variable=condition) 

### Intersect pairwise results with full results



In [ ]:
denrsboth={}
for i in mypairs:
    print(i[0]+' vs. '+i[1])
    DElist[i[0]+'-'+i[1]][i[0]].index=DElist[i[0]+'-'+i[1]][i[0]]['Name']
    DElist[i[0]+'-'+i[1]][i[1]].index=DElist[i[0]+'-'+i[1]][i[1]]['Name']
    bothDE1=DElist[i[0]+'-'+i[1]][i[0]].loc[list(set(DElist[i[0]+'-'+i[1]][i[0]]['Name']).intersection(set(DEgenes[i[0]]['Name']))),:]
    bothDE2=DElist[i[0]+'-'+i[1]][i[1]].loc[list(set(DElist[i[0]+'-'+i[1]][i[1]]['Name']).intersection(set(DEgenes[i[1]]['Name']))),:]
    print('DE.1')
    print(bothDE1.sort_values('Log2FC',ascending=False).iloc[0:20,:])
    print('DE.2')
    print(bothDE2.sort_values('Log2FC',ascending=False).iloc[0:20,:])
    denrsboth[i[0]+'-'+i[1]]=[len(bothDE1.index),len(bothDE2.index)]
    bothDE1.to_csv(results_out_cell+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-allANDpairwise-'+i[0]+'-'+i[1]+'.1.csv')
    bothDE2.to_csv(results_out_cell+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-allANDpairwise-'+i[0]+'-'+i[1]+'.2.csv')
denrsboth=pd.DataFrame(denrsboth)
denrsboth.index=['DE1','DE2']

In [ ]:
#DElist['C_24_Tumor-T_24_Tumor']['T_24_Tumor']

In [ ]:
plt.rcParams["figure.figsize"] = (4,4)
ax=denrsboth.transpose().plot.bar()
ax.set_ylabel("Nr. pairwise DE genes")

### Gene set enrichment analysis 

In [ ]:
import gseapy
#gseapy.get_library_name()

In [ ]:
dbs=['KEGG_2021_Human','KEGG_2019_Mouse','GO_Biological_Process_2021','Reactome_2022','Human_Gene_Atlas',
     'WikiPathways_2019_Mouse','NCI-Nature_2016',
     'ChEA_2022','CellMarker_Augmented_2021','Azimuth_Cell_Types_2021',
     'COVID-19_Related_Gene_Sets_2021','MSigDB_Hallmark_2020']


In [ ]:
cdata.raw.var['MeanExpr']=cdata.raw.X.mean(axis=0).tolist()[0]

In [ ]:
for i in mypairs:
    print(i[0]+' vs. '+i[1])
    DElist[i[0]+'-'+i[1]][i[0]].index=DElist[i[0]+'-'+i[1]][i[0]]['Name']
    DElist[i[0]+'-'+i[1]][i[1]].index=DElist[i[0]+'-'+i[1]][i[1]]['Name']
    #bothDE1=DElist[i[0]+'-'+i[1]][i[0]].loc[list(set(DElist[i[0]+'-'+i[1]][i[0]]['Name']).intersection(set(DEgenes[i[0]]['Name']))),:]
    #bothDE2=DElist[i[0]+'-'+i[1]][i[1]].loc[list(set(DElist[i[0]+'-'+i[1]][i[1]]['Name']).intersection(set(DEgenes[i[1]]['Name']))),:]
    bothDE1=DElist[i[0]+'-'+i[1]][i[0]].sort_values('Log2FC', ascending=False).iloc[0:200,:]
    bothDE2=DElist[i[0]+'-'+i[1]][i[1]].sort_values('Log2FC', ascending=False).iloc[0:200,:]
    for j in dbs:
        if (len(bothDE1['Name'].tolist())>10):
            gseapy.enrichr(gene_list=bothDE1['Name'].tolist(), organism='human', 
                   gene_sets=j, background=list(cdata.raw.var.loc[cdata.raw.var['MeanExpr']>0.01]['SYMBOL']),
                           cutoff=0.05, format='png',outdir=results_out_cell+'enrichr/'+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-pairwiseTop200-'+i[0]+'-'+i[1]+'_1/')
        if (len(bothDE2['Name'].tolist())>10):
            gseapy.enrichr(gene_list=bothDE2['Name'].tolist(),  organism='human', 
                   gene_sets=j, background=list(cdata.raw.var.loc[cdata.raw.var['MeanExpr']>0.01]['SYMBOL']),
                           cutoff=0.05, format='png',outdir=results_out_cell+'enrichr/'+'DE-wilcoxon-logFC'+myfcstr+'-padj'+mypvalstr+'-UP-'+mylevel+'-'+mysubset.replace(" ", "_")+'-pairwiseTop200-'+i[0]+'-'+i[1]+'_2/')

### Write out the fold-change for chosen genes


In [ ]:
sc.settings.set_figure_params()

In [ ]:
sc.pl.umap(cdata,color=['dblabel'])

In [ ]:
sc.pl.umap(cdata,color=['leiden']) ### cluster 38 vs. 26 may reflect the distinct cDC1 phenotype

In [ ]:
sc.pl.umap(cdata,color=['treatment_id']) ### cluster 38 vs. 26 may reflect the distinct cDC1 phenotype

In [ ]:
sc.pl.umap(cdata,color=['sample_id']) ### cluster 38 vs. 26 may reflect the distinct cDC1 phenotype

In [ ]:
sc.pl.umap(cdata,color=goiorder, color_map='viridis')

In [ ]:
sc.pl.umap(cdata,color=latei, color_map='viridis')

In [ ]:
! jupyter nbconvert --to html DEAnalysis-IFNGpos.ipynb